In [3]:
import skimage.io
import skimage.color
import skimage.transform
import matplotlib.pyplot as plt
%matplotlib inline

import numpy as np
import os
import random

import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

In [4]:
DATASET_DIR = '../dataset/carplate'

classes = os.listdir(DATASET_DIR + "/ann/")
data = []
for cls in classes:
    files = os.listdir(DATASET_DIR + "/ann/"+cls)
    for f in files:
        img = skimage.io.imread(DATASET_DIR + "/ann/"+cls+"/"+f)
        img = skimage.color.rgb2gray(img)
        data.append({
            'x': img,
            'y': cls
        })

random.shuffle(data)

In [6]:
X = [d['x'] for d in data]
y = [d['y'] for d in data]

ys = list(np.unique(y))
y = [ys.index(v) for v in y]

x_train = np.array(X[:int(len(X)*0.8)])
y_train = np.array(y[:int(len(X)*0.8)])

x_test = np.array(X[int(len(X)*0.8):])
y_test = np.array(y[int(len(X)*0.8):])

In [7]:
batch_size = 128
num_classes = len(classes)
epochs = 10

# input image dimensions
img_rows, img_cols = 20, 20

In [8]:
def extend_channel(data):
    if K.image_data_format() == 'channels_first':
        data = data.reshape(data.shape[0], 1, img_rows, img_cols)
    else:
        data = data.reshape(data.shape[0], img_rows, img_cols, 1)
        
    return data

x_train = extend_channel(x_train)
x_test = extend_channel(x_test)

input_shape = x_train.shape[1:]

In [9]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

y_train2 = keras.utils.to_categorical(y_train, num_classes)
y_test2 = keras.utils.to_categorical(y_test, num_classes)

x_train shape: (12920, 20, 20, 1)
12920 train samples
3231 test samples


In [10]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [11]:
model.fit(x_train, y_train2,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test2))

Train on 12920 samples, validate on 3231 samples
Epoch 1/10
12920/12920 [==============================] - 2s 179us/step - loss: 2.2041 - acc: 0.4952 - val_loss: 0.7677 - val_acc: 0.8285
Epoch 2/10
12920/12920 [==============================] - 1s 59us/step - loss: 0.8010 - acc: 0.7997 - val_loss: 0.3546 - val_acc: 0.9171
Epoch 3/10
12920/12920 [==============================] - 1s 71us/step - loss: 0.4948 - acc: 0.8752 - val_loss: 0.2498 - val_acc: 0.9474
Epoch 4/10
12920/12920 [==============================] - 1s 56us/step - loss: 0.3751 - acc: 0.9078 - val_loss: 0.2035 - val_acc: 0.9579
Epoch 5/10
12920/12920 [==============================] - 1s 55us/step - loss: 0.3107 - acc: 0.9214 - val_loss: 0.1881 - val_acc: 0.9582
Epoch 6/10
12920/12920 [==============================] - 1s 58us/step - loss: 0.2740 - acc: 0.9335 - val_loss: 0.1711 - val_acc: 0.9660
Epoch 7/10
12920/12920 [==============================] - 1s 56us/step - loss: 0.2477 - acc: 0.9375 - val_loss: 0.1522 - val_acc

In [12]:
score = model.evaluate(x_test, y_test2, verbose=0)

print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.13915864136
Test accuracy: 0.971216341708


In [13]:
model.save_weights('char_cnn.h5')